Introduction
=======

Expressions in **Prove-It** are fundamental and versatile.  In order to prove some statement, you must be able to express the statement that you want to prove, and express each axiom and each theorem that is used to construct the proof as well as intermediate statements along the way.  Statements and proofs will be discussed in later tutorials.  Here we focus on the expressions themselves.  In **Prove-It**, an expression is a tree-like data structure.  As a Python object, its base class is **proveit.core.expr.Expression** (also aliased as proveit.Expression via "from ... import ..." statements within **\_\_init\_\_.py** files).  It must have an intermediate class in the class inheritance hierarchy that is one of the "core" **Expression** classes which governs how the expression behaves when used in a proof (mainly governing the **specialize** derivation step which will be discussed in subsequent tutorials).  

In the **Prove-It** philosophy, being explicit is valued, to an extreme, over being implicit.  To this end, the internal representation of an expression should directly translate to the external representation of how the expression is to be displayed.  This way, there is no ambiguity and no confusion, and all transformations regarding equivalent ways of expressing a statement can be traced to explicit axioms.  As a consequence, however, **Prove-It** is not a minimalist approach with respect to types of expressions or with respect to numbers of axioms.  One example is currying.  Haskell Curry showed a minimalistic way to avoid multi-argument functions by expanding them as nested single-argument functions; however, the **Prove-It** philosophy is to be explicit about using multi-argument operations when one wishes to express statements in that form.

The following are the core **Expression** classes that enable a variety of explicit forms of expressions.  A brief description is provided for each class, but these concepts will become more clear by considering examples shown below.
* **Label**: A label for a mathematical object (e.g., a symbol).
    * **Variable**: A label with no contextual meaning.
    * **MultiVariable**: A placeholder for multiple **Variable**s, indicated by displaying one or more dummy, symbolic tensor-like indices.
    * **Literal**: A label with contextual meaning that is bound to some *context* (specified as a Python package).
* **Lambda**: Represents a mathematical mapping or function; contains the *arguments* (an **ExpressionList** of **Variable**s or **MultiVariable** **Bundle**s that are being mapped) and an *expression* (any **Expression** that the arguments are to be mapped into).
* **Operation**: Represents an applied operation; contains an *operator* (a **Label** or **Lambda**) and *operands* (a **Composite**).
* **Composite**: An **Expression** that contains multiple **Expression**s in a collection.
    * **ExpressionList**: A 1-dimensional collection of **Expressions**s.
    * **ExpressionTensor**: A multi-dimensional collection of **Expression**s.
    * **NamedExpressions**: Maps keyword strings into **Expression**s.
* **Bundle**: A placeholder for a portion of a **Composite**. 
    * **Etcetera**: A placeholder for a portion of an **ExpressionList**.
    * **Block** A placeholder for a portion of an **ExpressionTensor**.

Developers and users may derive from core **Expression** classes to make their own **Expression** classes that have special formatting rules and methods for manipulating and utilizing the expression.  The formatting should be in direct correspondence with the structure of the **Expression** in regards to core **Expression** classes, but there is no enforcement mechanism to ensure that this is the case.  Users are advised to inspect the core-defined structure of **Expression**s to ensure that the formatting properly reflects the internal content (or rely on crowd-source checking of axioms/theorems).

Labels: Variable, MultiVariable, and Literal
================================

In [1]:
from proveit import Label, Variable, MultiVariable, Literal

Label
-----

A **Label** is not an **Expression** but is the super class for the **Variable**, **MultiVariable**, and **Literal** **Expression** classes.

In [2]:
# A Label has a string format and a latex format
triangle = Label(stringFormat='triangle', latexFormat=r'\triangle')
print "'triangle' string format:", triangle.string()
print "'triangle' latex format:", triangle.latex()
print "IPython display of 'triangle':"
triangle # the latex is processed and displayed in IPython

'triangle' string format: triangle
'triangle' latex format: \triangle
IPython display of 'triangle':


Variable
--------

In [3]:
# These are Variables which are interchangeable labels with no formal, contextual meaning
x = Variable('x') # the string and latex formats are the same by default
x

In [4]:
omega = Variable(stringFormat='omega', latexFormat=r'\omega') # different string and latex format
omega 

As a convention, convenient Variables and Expressions are contained in **common** modules in various packages.  That is, **common** modules hold common expressions:

In [25]:
from proveit.common import x, y, z, alpha
alpha

MultiVariable
-------------

In [5]:
# A MultiVariable represents multiple Variables with one Label
r = MultiVariable('r', latexFormat=r'{\vec r}')
r # has a dummy index (box by default) to indicate that it can be expanded to a list of Variables

In [6]:
# Can also be used to represent a multi-dimensional array of Variables
A = MultiVariable('A', numDimensions=2)
A # a box and triangle are used as the multi-dimensional dummy indices

In [7]:
# The default dummy indices may be overridden
B = MultiVariable('B', numDimensions=2, axesLabels=[Label('@', r'\triangle'), Label('*', r'\Box')])
B # switched the triangle and box just to show we can

Examples of substituting **MultiVariable**s for multiple **Variable**s will be given in the **Bundle** section.

Literal
-------

A **Literal** is another kind of **Label**.  In contrast to **Variable**s, **Literal**s have a formal, contextual meaning. They are bound to a *context* (as a Python package) and endowed with meaning via axioms as will be discussed later.  Below are examples of where a **Literal** could appropriately be used, where the context is set as the tutorial package. 

### Specific operators

In [8]:
TIMES = Literal(context='proveit.tutorial', stringFormat='*', latexFormat=r'\times')
TIMES # By convention, Literal operators are ALL-CAPS

In [9]:
FACTORIAL = Literal('proveit.tutorial', '!')
FACTORIAL

In [10]:
SUMMATION = Literal('proveit.tutorial', 'sum', r'\sum')
SUMMATION

### Constant values

In [11]:
FIVE = Literal('proveit.tutorial', '5')
FIVE

In [12]:
TRUE = Literal('proveit.tutorial', 'true', r'\top')
TRUE # By convention, Literal constants are also ALL-CAPS

In [13]:
FALSE = Literal('proveit.tutorial', 'false', r'\bot')
FALSE

Note that it may be desirable to derive a class from **Literal** to use as the class for some constant values.  For example, **proveit.TRUE** and **proveit.FALSE** (an alias of **proveit.contexts.logic.boolean.boolSet.TRUE** and **proveit.contexts.logic.boolean.boolSet.FALSE** respectively) are of type **proveit.contexts.logic.boolean.boolSet.TrueLiteral** and **proveit.contexts.logic.boolean.boolSet.FalseLiteral** respectively.  These classes have the method **evalEquality** that is convenient for invoking $\top = \top$, $\bot = \bot$, $\top \neq \bot$, or $\bot \neq \top$ as desired:

In [14]:
from proveit.logic import TRUE
TRUE.evalEquality(FALSE)

ImportError: No module named expression.Expression

### Contextual "variables"

This is appropriate when variables are given a specific meaning within the context of a problem.  For example, consider a story math problem where Andrea has $a$ apples and Bill has $b$ bananas.  These are variables in a sense, but here they have contextual meaning so they should be **Literal**s.  However, suppose one wants to prove a general theorem for any number of Andrea's apples and Bill's bananas.  One may start by using the $a$ and $b$ **Literal**s for convenience for some contextual "lemmas" (although ProveIt does not distinguish between lemmas and theorems), but then demote them to **Variable**s in order to make a statement of the form $\forall_{a, b} P(a, b)$ (**Literal**s may not be quantified over).  This is done by using a process called axiom elimination that will be discussed later.  For now, we just note that "variables" with contextual meaning need to be **Literal**s.

In [ ]:
aLit = Literal('proveit.tutorial', 'a')
aLit

In [ ]:
bLit = Literal('proveit.tutorial', 'b')
bLit

There is a potential source of confusion when a **Literal** and a **Variable** are displayed in the same way.  Our solution is for **Prove-It** to check for such mistakes and raise appropriate exceptions that alert the user to the problem.  For example, users will be alerted when attempting to quantify over a **Literal** that there may be such a mix up.  Also, when equivalence is checked between **Expression**s, we check the equivalence when all **Literal**s are converted to their corresponding **Variable**s (with the same string and latex format) to see if that changes the conclusion regarding equivalence. 

Lambda
====

Represents a mathematical mapping or function; contains the *arguments* (an **ExpressionList** of **Variable**s or **MultiVariable** **Bundle**s that are being mapped) and an *expression* (any **Expression** that the arguments are to be mapped into).

In [15]:
from proveit import Lambda

In [20]:
# maps any value to 5
Lambda(x, FIVE) 

In [16]:
# Can have multiple arguments, and the lambda's expression may involve any of these arguments
Lambda([x, omega], x)

In [18]:
# Can include MultiVariable Bundles
from proveit import Etcetera # More on Etcetera in the Bundle section
Lambda([x, Etcetera(r), omega], omega)

Operation
=========

Represents an applied operation; contains an operator (a Label or Lambda) and operands (a Composite).

In [21]:
from proveit import Operation

In [24]:
# Simple case: a Variable operator with a single Variable operand
f = Variable('f')
fx = Operation(f, x)
fx # f(x)

Common forms of these are available from **proveit.common**:

In [28]:
from proveit.common import fx, gx
gx

In [37]:
# Let's derive a class from Operation for a kind of operation with a specific literal operand.
class Factorial(Operation):
    
    def __init__(self, operand):
        # creates the Operation with FACTORIAL as the operator and the provided operand as its only operand.
        Operation.__init__(self, FACTORIAL, operand)
        self.operand = operand # store this here for convenience
        
    def string(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.string(fence=True) + self.operator.string()
    
    def latex(self, **kwargs): # should accept kwargs even when not used (e.g., 'fence')
        # the operand should be fenced (wrapped in parentheses) to prevent ambiguity
        return self.operand.latex(fence=True) + self.operator.latex() 

In [36]:
# Now we can make
Factorial(FIVE)

A **Factorial** is an **Operation** with **FACTORIAL** as the **Literal** operator.  Its formatting is altered from the default but is still true (in a 1-to-1 correspondence) to the core **Expression** structure.

In [38]:
# show core structure

Here, in the IPython notebook setting, we just need the latex formatting, but it is always a good idea to define the string formatting for whenever it is needed.  The `fence=True` above indicates that parentheses should be used when it could be ambiguous otherwise.  Our example above has no ambiguity, so parentheses are not used.  We will show a case below where the parentheses are required.

Now let's consider a multiple operand example.  Internally, this uses an **ExpressionList** that will be discussed in the **Composite** section.  Rather than derictly deriving the class from **Operation**, we'll use **AssociativeOperation** which has default formatting that is applicable for our needs here.

In [43]:
from proveit.logic.genericOps import AssociativeOperation

class Multiply(AssociativeOperation):
    def __init__(self, *operands): # takes a list of arguments as the operands
        # creates the AssociativeOperation with TIMES as the operator and any number of operands.
        AssociativeOperation.__init__(self, TIMES, *operands)

In [46]:
# Demonstrating an Operation with multiple operands
multExpr = Multiply(x, FIVE, omega)
multExpr

In [47]:
# Let's nest Operations and show proper fencing behavior
Factorial(multExpr)

TODO: show structure.
Also need to do an example with a Lambda operator